In [5]:
import pandas as pd 
import numpy as np
import PyPDF2

import re
import spacy
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction import text
from nltk.tokenize import RegexpTokenizer

import glob # Package for Unix-style pathname pattern expansion
import os   # Python operating system interface

import nltk
from nltk.text import Text

import json

In [2]:
#tokenize 

def get_tokens(path):
    
    #read pdf file
    pdf=open(path, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf)
    number_of_pages = read_pdf.getNumPages()
    un_documents=[]
    for i in range(number_of_pages):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        un_documents.append(page_content)

    #tokenization + change to lower case
    tokenizer = RegexpTokenizer('\w+')
    tokens_set=[]
    global tokens
    tokens=[]
    for i in un_documents:
        tokens_set.append(tokenizer.tokenize(i))

    for i in range(len(tokens_set)):
        for j in range(len(tokens_set[i])):
            tokens_set[i][j]=tokens_set[i][j].lower()
            tokens.append(tokens_set[i][j])
            
    tokens= [word for word in tokens if len(word) > 3]
    
    #lemmatizer  *Lemmatisation (or lemmatization) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    #stemmer = nltk.stem.PorterStemmer()
    #tokens = [stemmer.stem(t) for t in tokens]
    
    
    #delete stopwords 
    tokens= [word for word in tokens if len(word) > 2]
    sp = spacy.load('en_core_web_sm')
    all_stopwords = sp.Defaults.stop_words
    all_stopwords.update(["the","and","that","is","are","was","were","for","mr","mrs"])
    all_stopwords.update(["united","nation","international","country", "world", "state", "including", "excluding", "like", "meeting", "believe", "year","general","assembly","session","president","regard","development","security","peace","people","republic","council","europeon","union","prime","minister","foreign","affair","look","forward","lady","gentleman","september","spoke","sixty","seventy","secretary"])
    tokens= [word for word in tokens if not word in all_stopwords]
    #remove numbers
    
    tokens=[word for word in tokens if re.match('^[a-zA-Z]+', word)]
    
    
    return tokens

In [ ]:
sub_folders=[]
 
rootdir = "C:\\project\\UN\\UN_pdfs"
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        sub_folders.append(d)

        
for i in sub_folders:
    path ="C:\\project\\UN\\UN_pdfs\\"+i
    year=path[-4:]
    print(year)
    file_list = []

    for root, dirs, files in os.walk(path):
        for file in files:
            #append the file name to the list
            file_list.append(os.path.join(root,file))

    print("number of files:",len(file_list))



    total_tokens=[]

    for i in file_list:
        #get tokens from files
        get_tokens(i)
        total_tokens.extend(tokens)
    print("the number of tokens:",len(total_tokens))
    
    token_save.update({year:total_tokens})
    
with open('data.json', 'w') as fp:
    json.dump(token_save, fp, indent=4)